# Generate Dataset

This notebook geenrates a dataset for a 4 DoF spatial robotic arm described below:
- 4 revolute joints with no limits
- The following limits on each joint in radians: `[(-1, 1), (-0.8, 0.8), (-0.8, 0.8), (-0.8, 0.8)]`
- 5 links of the following sizes: `[0.2, 0.2, 0.5, 0.5, 0.2]`

Our dataset consists of a set of features, $X$, and a set of targets $y$.
- $X$ consists of a 7 values. The first 3 represent a point $(x,y,z)$ and the last 4 represent an orientation in the form of a quaternion $(x,y,z,w)$
- $y$ consists of joint angles, with the amount of features being equal to the amount of joints.
- Additonally the pose and orientation or each joint is added to the dataset.

### Adjust Path
In order to use the RobotArm class located in `inverse_kinematics_prediction/utils/robot_arm.py` we need to add this project to our system path.

In [6]:
import sys
import os

# Add the parent directory (project root) relative to the current working directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Current working directory:", os.getcwd())
print("Project root added to sys.path:", project_root)

Current working directory: c:\Users\cjsta\git\inverse_kinematics_prediction\notebooks
Project root added to sys.path: c:\Users\cjsta\git\inverse_kinematics_prediction


### Create a RobotArm object
The RobotArm object defines the structure of a desired arm and can calculate the location of it's end effector.

In [7]:
import numpy as np

from utils.robot_arm import RobotArm

# Create a 4-DOF robot arm with 5 links, each of length 1.0 unit.
num_joints = 4
joint_limits = [(-1*np.pi, 1*np.pi), (-0.8*np.pi, 0.8*np.pi), (-0.8*np.pi, 0.8*np.pi), (-0.8*np.pi, 0.8*np.pi)]
rotation_axes = ['z', 'y', 'y', 'y']
link_lengths = [0.2, 0.2, 0.5, 0.5, 0.2]
link_axes = ['z', 'z', 'z', 'z', 'z']

# Instantiate the RobotArm object
robot_arm = RobotArm(num_joints=num_joints, 
                     joint_limits=joint_limits, 
                     link_lengths=link_lengths, 
                     rotation_axes=rotation_axes,
                     link_axes=link_axes)

# Test forward kinematics
test_angles = robot_arm.sample_random_joint_angles()
pos, quat = robot_arm.forward_kinematics(test_angles)
print("Joint angles (radians): ", test_angles)
print("End Effector Position (point x,y,z): ", pos)
print("End Effector Orientation (quaternion x,y,z,w): ", quat)

Joint angles (radians):  [ 2.65252784  1.5159563  -0.75143509  1.74329513]
End Effector Position (point x,y,z):  [-0.85080076  0.45278552  0.62710381]
End Effector Orientation (quaternion x,y,z,w):  (np.float64(0.9219415685560793), np.float64(-0.23004830313745706), np.float64(-0.3023409379348281), np.float64(-0.07544189579153755))


### Generate Targets
Our target is the position and orientation of the end effector. We generate a uniformly random set of those. 

In [8]:
num_samples = 100000  # Parameter to control the number of samples

joint_samples = np.zeros((num_samples, robot_arm.num_joints))

# Generate random angles for each joint within its allowed range.
for i, (low, high) in enumerate(robot_arm.joint_limits):
    joint_samples[:, i] = np.random.uniform(low, high, size=num_samples)

print("Generated random joint samples shape:", joint_samples.shape)

Generated random joint samples shape: (100000, 4)


### Generate Features
Our features are the position and orientation of the end effector.



In [9]:
# Preallocate lists to store positions and quaternions for all joints
joint_data = []

# Loop through each sample and compute the joint poses
for joint_angles in joint_samples:
    # Get poses for all joints including base and end effector
    poses = robot_arm.get_joint_poses(joint_angles)
    
    # Skip base (index 0) and collect data for all joints and end effector
    sample_data = []
    
    # Add joint angles first
    sample_data.extend(joint_angles)
    
    # Add position and orientation for each joint (skip base at index 0)
    for i in range(1, len(poses)):
        pos, quat = poses[i]
        sample_data.extend(pos)        # Add position [x, y, z]
        sample_data.extend(quat)       # Add quaternion [x, y, z, w]
    
    joint_data.append(sample_data)

# Convert the list to a numpy array
joint_samples_data = np.array(joint_data)

print("Joint data shape:", joint_samples_data.shape)

Joint data shape: (100000, 39)


### Save Dataset

In [10]:
import pandas as pd

# Create column names
columns = []

# Joint angle columns
for i in range(robot_arm.num_joints):
    columns.append(f'joint_{i+1}_angle')

# For each joint (skip base), add position and orientation columns
for i in range(1, robot_arm.num_joints + 2):  # +2 because we have num_joints+1 links (including end effector)
    if i <= robot_arm.num_joints:
        prefix = f'joint_{i}'
    else:
        prefix = 'end_effector'
    
    # Add position columns
    columns.extend([f'{prefix}_pos_x', f'{prefix}_pos_y', f'{prefix}_pos_z'])
    
    # Add orientation columns
    columns.extend([f'{prefix}_quat_x', f'{prefix}_quat_y', f'{prefix}_quat_z', f'{prefix}_quat_w'])

# Create a DataFrame with the data and the dynamically generated column names
df = pd.DataFrame(joint_samples_data, columns=columns)

# Create the target directory if it doesn't exist
target_dir = os.path.join(project_root, "data", "raw")
os.makedirs(target_dir, exist_ok=True)

# Define the file path
file_path = os.path.join(target_dir, "dataset.csv")
df.to_csv(file_path, index=False)

print("CSV file saved as", file_path)
print("Column names:", columns)

CSV file saved as c:\Users\cjsta\git\inverse_kinematics_prediction\data\raw\dataset.csv
Column names: ['joint_1_angle', 'joint_2_angle', 'joint_3_angle', 'joint_4_angle', 'joint_1_pos_x', 'joint_1_pos_y', 'joint_1_pos_z', 'joint_1_quat_x', 'joint_1_quat_y', 'joint_1_quat_z', 'joint_1_quat_w', 'joint_2_pos_x', 'joint_2_pos_y', 'joint_2_pos_z', 'joint_2_quat_x', 'joint_2_quat_y', 'joint_2_quat_z', 'joint_2_quat_w', 'joint_3_pos_x', 'joint_3_pos_y', 'joint_3_pos_z', 'joint_3_quat_x', 'joint_3_quat_y', 'joint_3_quat_z', 'joint_3_quat_w', 'joint_4_pos_x', 'joint_4_pos_y', 'joint_4_pos_z', 'joint_4_quat_x', 'joint_4_quat_y', 'joint_4_quat_z', 'joint_4_quat_w', 'end_effector_pos_x', 'end_effector_pos_y', 'end_effector_pos_z', 'end_effector_quat_x', 'end_effector_quat_y', 'end_effector_quat_z', 'end_effector_quat_w']
